In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sympy as sp
import pyPI as pi

In [2]:
# Definir vectores iniciales
Er_1 = np.array([[1], [0]])  
Er_2 = np.array([[1], [1]])  
Es = np.array([[1], [2 * np.exp(1j * np.pi/4)]])

# Calcular intensidades
info_1, i_1 = pi.I(Er_1, Es)
info_2, i_2 = pi.I(Er_2, Es)

# Cálculo de componentes de Es
Esx = info_1[1] / np.abs(Er_1[0])
Esy = np.sqrt(info_1[0] - Esx**2 - Er_1[0]**2)
Es_c = np.array([[Esx], [Esy]])

# Cálculo del numerador y denominador
norm_Er2_sq = np.linalg.norm(Er_2)**2
norm_Es_c_sq = np.linalg.norm(Es_c)**2

numerador = (norm_Er2_sq + norm_Es_c_sq) * np.tan(info_2[2])[0]
denominador = norm_Er2_sq + norm_Es_c_sq + 2 * np.abs(Er_2[0])[0] * np.abs(Es_c[0])[0] * np.abs(Er_2[1])[0] * np.abs(Es_c[1])[0]

# Definir variable simbólica
phi_s = sp.Symbol('Delta_phi_s')

# Definir matrices simbólicas
Es_c_sp = sp.Matrix([Esx[0], Esy[0]])  
Er_2_sp = sp.Matrix([Er_2[0,0], Er_2[1,0]]) 

# Expresión para la fase
expr = sp.atan2(-Er_2_sp[1] * Es_c_sp[1] * sp.sin(-phi_s), 
                Er_2_sp[0] * Es_c_sp[0] + Er_2_sp[1] * Es_c_sp[1] * sp.cos(-phi_s)) - info_2[2]

# Resolver para Δφ_s
solucion = sp.solve(expr, phi_s)
delta_phi_s = solucion[0] if solucion else "No se encontraron soluciones simbólicas."

# Resultados
print(f"Esx: {Esx}")
print(f"Esy: {Esy}")
print(f"Delta_phi_s: {delta_phi_s}")
print(f"Ángulo de Es[1]: {np.angle(Es[1,0])}")

Esx: [1.]
Esy: [2.]
Delta_phi_s: (0.785398163397449,)
Ángulo de Es[1]: 0.7853981633974483


In [ ]:
def Es_parameters(info_1, info_2, p=False):
    """
    Computes the parameters Esx, Esy, and Delta_phi_s for a given electric field vector Es.

    Parameters:
    Es (numpy array): 2x1 complex electric field vector.
    p (bool, optional): If True, prints the calculated parameters. Default is False.

    Returns:
    tuple: (Esx, Esy, Delta_phi_s) where:
        - Esx: x-component of the estimated electric field.
        - Esy: y-component of the estimated electric field.
        - Delta_phi_s: Phase shift between components.
    """

    # Define reference fields
    Er_1 = np.array([[1], [0]])  # Reference vector 1
    Er_2 = np.array([[1], [1]])  # Reference vector 2

    # Calculate Esx and Esy
    Esx = info_1[1] / np.abs(Er_1[0])
    Esy = np.sqrt(info_1[0] - Esx**2 - Er_1[0]**2)
    Es_c = np.array([[Esx], [Esy]])  # Construct estimated field vector

    # Compute squared norms of reference and estimated fields
    norm_Er2_sq = np.linalg.norm(Er_2)**2
    norm_Es_c_sq = np.linalg.norm(Es_c)**2
    
    # this inot working by coding but by handmade
    # Check later
    # Compute numerator and denominator for phase calculation
    # numerator = (norm_Er2_sq + norm_Es_c_sq) * np.tan(info_2[2])[0]
    # denominator = norm_Er2_sq + norm_Es_c_sq + 2 * np.abs(Er_2[0])[0] * np.abs(Es_c[0])[0] * np.abs(Er_2[1])[0] * np.abs(Es_c[1])[0]
    # delta_phi_s = np.arctan2(numerator, denominator)

    # Define symbolic variable for phase shift
    phi_s = sp.Symbol('Delta_phi_s')

    # Define symbolic matrices for field components
    Es_c_sp = sp.Matrix([Esx[0], Esy[0]])  # Symbolic estimated field vector
    Er_2_sp = sp.Matrix([Er_2[0,0], Er_2[1,0]])  # Symbolic reference vector

    # Expression for phase shift calculation
    expr = sp.atan2(-Er_2_sp[1] * Es_c_sp[1] * sp.sin(-phi_s), 
                    Er_2_sp[0] * Es_c_sp[0] + Er_2_sp[1] * Es_c_sp[1] * sp.cos(-phi_s)) - info_2[2]

    # Solve for Delta_phi_s
    solution = sp.solve(expr, phi_s)
    delta_phi_s = solution[0] if solution else "No symbolic solution found."

    # Convert Esx and Esy to scalar values
    Esx = Esx[0]
    Esy = Esy[0]
    delta_phi_s = delta_phi_s[0] if isinstance(delta_phi_s, (list, tuple)) else delta_phi_s

    # Print results if required
    if p:
        print("Calculated Parameters:")
        print(f"Esx: {Esx}")
        print(f"Esy: {Esy}")
        print(f"Delta_phi_s: {delta_phi_s}")
    # Return computed values
    return Esx, Esy, delta_phi_s


In [4]:
Er = np.array([[1], [0]])
Eis = np.array([[1], [2 * np.exp(1j * np.pi/4)]])
#Eis = np.array([[1], [1]])

delta_chi = np.pi
alpha = 0

#S = np.array([[1, 0], 
#                   [0, np.exp(1j * delta_chi)]])
sxx = (np.cos(alpha) ** 2 + np.exp(-1j * delta_chi) * np.sin(alpha) ** 2)
sxy = ((1 - np.exp(1j * delta_chi)) * np.cos(alpha) * np.sin(alpha))
syx = ((1 - np.exp(1j * delta_chi)) * np.cos(alpha) * np.sin(alpha))
syy = (np.sin(alpha) ** 2 + np.exp(-1j * delta_chi) * np.cos(alpha) ** 2)

S = np.array([[sxx, sxy], 
                   [syx, syy]])

Es = S @ Eis

delta_phi_s = np.angle(Es[1,0]) - np.angle(Es[0,0])

delta_chi_calculated = np.angle(Eis[1,0]) -delta_phi_s


print(f"S: {S}")
print(f"Es: {Es}")
print(f"Esx: {np.abs(Es[0])[0]}")
print(f"Esy: {np.abs(Es[1])[0]}")
print(f"Delta_phi_s_x: {np.angle(Es[0,0])}")
print(f"Delta_phi_s_y: {np.angle(Es[1,0])}")
print(f"Delta_phi_s: {delta_phi_s}")
print(f"Delta_phi_is: {np.angle(Eis[1,0])}")
print(f"Delta_chi: {delta_chi}")
print(f"Delta_chi_calculates: {delta_chi_calculated}")

S: [[ 1.+0.0000000e+00j  0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j -1.-1.2246468e-16j]]
Es: [[ 1.        +0.j        ]
 [-1.41421356-1.41421356j]]
Esx: 1.0
Esy: 2.0000000000000004
Delta_phi_s_x: 0.0
Delta_phi_s_y: -2.356194490192345
Delta_phi_s: -2.356194490192345
Delta_phi_is: 0.7853981633974483
Delta_chi: 3.141592653589793
Delta_chi_calculates: 3.141592653589793


In [4]:
import sympy as sp

# Definir variables simbólicas
vartheta = sp.Symbol('vartheta')  
phi_s = sp.Symbol('Delta_phi_s')  # Se especifica que es real

# Definir elementos simbólicos de las matrices
Es_x, Es_y = sp.symbols('Es_x Es_y')
Er_x, Er_y = sp.symbols('Er_x Er_y')

# Definir expresiones para seno y coseno de phi_s
sin_phi_s = sp.sin(-phi_s)
cos_phi_s = sp.cos(-phi_s)

# Expresión en términos de tangente
tan_expr = (-Er_y * Es_y * sin_phi_s) / (Er_x * Es_x + Er_y * Es_y * cos_phi_s)

# Igualamos la tangente con tan(vartheta)
expr = sp.tan(vartheta) - tan_expr

# Resolver para phi_s
solution = sp.solve(expr, phi_s)

# Imprimir solución simbólica
print(solution if solution else "No se encontraron soluciones simbólicas.")

[2*atan((Er_y*Es_y - sqrt(-Er_x**2*Es_x**2*tan(vartheta)**2 + Er_y**2*Es_y**2*tan(vartheta)**2 + Er_y**2*Es_y**2))/((Er_x*Es_x - Er_y*Es_y)*tan(vartheta))), 2*atan((Er_y*Es_y + sqrt(-Er_x**2*Es_x**2*tan(vartheta)**2 + Er_y**2*Es_y**2*tan(vartheta)**2 + Er_y**2*Es_y**2))/((Er_x*Es_x - Er_y*Es_y)*tan(vartheta)))]


In [1]:
import sympy as sp

# Definir las variables simbólicas
alpha = sp.Symbol('alpha')
Delta_phi_sx, Delta_phi_sy, Delta_phi_is = sp.symbols('Delta_phi_sx Delta_phi_sy Delta_phi_is')
Delta_chi = sp.Symbol('Delta_chi')
i = sp.I  # Unidad imaginaria

A_sx, A_sy, A_isx, A_isy = sp.symbols('A_sx A_sy A_isx A_isy')

# Definir los términos Sxx, Sxy, Syx, Syy
S_xx = (sp.cos(alpha)**2 + sp.exp(-i * Delta_chi) * sp.sin(alpha)**2) * A_isx
S_xy = ((1 - sp.exp(i * Delta_chi)) * sp.cos(alpha) * sp.sin(alpha)) * A_isy * sp.exp(i * Delta_phi_is)
S_yx = ((1 - sp.exp(i * Delta_chi)) * sp.cos(alpha) * sp.sin(alpha)) * A_isx
S_yy = (sp.sin(alpha)**2 + sp.exp(-i * Delta_chi) * sp.cos(alpha)**2) * A_isy * sp.exp(i * Delta_phi_is)

# Definir las ecuaciones
eq1 = sp.Eq(A_sx * sp.exp(i * Delta_phi_sx), S_xx + S_xy)
eq2 = sp.Eq(A_sy * sp.exp(i * Delta_phi_sy), S_yx + S_yy)

# Resolver para alpha
solution = sp.solve([eq1, eq2], alpha)

# Mostrar la solución
display(solution)

KeyboardInterrupt: 